<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [3]:
import os
import pandas as pd
import json
import tweepy
import re 
import regex
import emoji
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

import scipy.cluster.hierarchy as sch
import numpy as np

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [11]:
# Leer las llaves
keys = pd.read_csv("claves.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [12]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [16]:
tweets = tweepy.Cursor(api.search,
                       q=['reina del flow -RT'],
                       lang='es',
                       geocode='4.3646,-74.0414,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)


In [17]:
# Objeto generador
t = [tweet for tweet in tweets]
t_processed = []

for tweet in t:
    print (tweet.full_text)
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    

@MarceloMejia Que bien. Y la reina del flow? 

https://t.co/YLhSYjCaFQ
Es normal tener pegadas las canciones de la reina del Flow??
Carolina Ramírez se confesó y reveló si ha visto con deseo a Juan Manuel Restrepo, su hijo en ‘La reina del flow 2’ https://t.co/XHtCzLIciE
@muchotropico La reina del flow?
La verdad no le tenia fe a la reina del flow 2. Anyways, PORQUE TÚ ERES MI OBSESIÓN 🔪🔪🔪
Caracol tv con su novela la reina del flow, transmite un mensaje absurdo sobre la juventud colombiana. No todos son antioqueños, cuyos ídolos son de los peores delincuentes que en esas tierras se han parido; ni aspiran a ser narcotraficantes, corruptos, sicarios, o prostitutas
@Krl77777 Que tiene que ver la reina del flow?
Gracias a mis parceros del @ElUniversalCtg la estamos vacilando con #lareinadelflow2 

https://t.co/eRSefspdeD
Top Series Netflix 🇨🇴 Mie19May

1. Pablo Escobar, el patrón del mal⬆️1
2. El inocente⬇️ 1
3. Sexify⬆️1
4. La reina del flow⬆️2
5. El legado de Jupiter⬇️2
6. Chica Vampiro⬆

In [18]:
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'fecha', 'ciudad', 'localizacion', 'id', 'enlace']
data.head()

,tweets,fecha,ciudad,localizacion,id,enlace
0,@MarceloMejia Que bien. Y la reina del flow? \...,2021-05-20 17:48:43,Bogotá,Girardot Cundinamarca,1395436339328430080,https://twitter.com/i/web/stuatus/139543633932...
1,Es normal tener pegadas las canciones de la re...,2021-05-20 15:10:53,Bogotá,Bogota,1395396619437215750,https://twitter.com/i/web/stuatus/139539661943...
2,Carolina Ramírez se confesó y reveló si ha vis...,2021-05-20 08:49:40,Bogotá,Bogotá,1395300682958245895,https://twitter.com/i/web/stuatus/139530068295...
3,@muchotropico La reina del flow?,2021-05-20 02:52:22,Bogotá,"Bogotá, D.C., Colombia",1395210765121036290,https://twitter.com/i/web/stuatus/139521076512...
4,La verdad no le tenia fe a la reina del flow 2...,2021-05-20 02:33:31,Bogotá,"Bogotá, D.C., Colombia",1395206021812047875,https://twitter.com/i/web/stuatus/139520602181...


In [19]:
import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions
  
  
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data


,tweets,fecha,ciudad,localizacion,id,enlace,texto_limpio,links,hashtags,mentions
0,@MarceloMejia Que bien. Y la reina del flow? \...,2021-05-20 17:48:43,Bogotá,Girardot Cundinamarca,1395436339328430080,https://twitter.com/i/web/stuatus/139543633932...,Que bien. Y la reina del flow? \n\n,[https://t.co/YLhSYjCaFQ],[],[@MarceloMejia]
1,Es normal tener pegadas las canciones de la re...,2021-05-20 15:10:53,Bogotá,Bogota,1395396619437215750,https://twitter.com/i/web/stuatus/139539661943...,Es normal tener pegadas las canciones de la re...,[],[],[]
2,Carolina Ramírez se confesó y reveló si ha vis...,2021-05-20 08:49:40,Bogotá,Bogotá,1395300682958245895,https://twitter.com/i/web/stuatus/139530068295...,Carolina Ramírez se confesó y reveló si ha vis...,[https://t.co/XHtCzLIciE],[],[]
3,@muchotropico La reina del flow?,2021-05-20 02:52:22,Bogotá,"Bogotá, D.C., Colombia",1395210765121036290,https://twitter.com/i/web/stuatus/139521076512...,La reina del flow?,[],[],[@muchotropico]
4,La verdad no le tenia fe a la reina del flow 2...,2021-05-20 02:33:31,Bogotá,"Bogotá, D.C., Colombia",1395206021812047875,https://twitter.com/i/web/stuatus/139520602181...,La verdad no le tenia fe a la reina del flow 2...,[],[],[]
5,"Caracol tv con su novela la reina del flow, tr...",2021-05-20 02:18:34,Bogotá,Circasia,1395202255738003457,https://twitter.com/i/web/stuatus/139520225573...,"Caracol tv con su novela la reina del flow, tr...",[],[],[]
6,@Krl77777 Que tiene que ver la reina del flow?,2021-05-20 01:21:25,Bogotá,"Bogotá, D.C., Colombia",1395187877454270464,https://twitter.com/i/web/stuatus/139518787745...,Que tiene que ver la reina del flow?,[],[],[@Krl77777]
7,Gracias a mis parceros del @ElUniversalCtg la ...,2021-05-20 00:43:44,Bogotá,"Bogotá, D.C., Colombia",1395178390408384520,https://twitter.com/i/web/stuatus/139517839040...,Gracias a mis parceros del la estamos vacilan...,[https://t.co/eRSefspdeD],[#lareinadelflow2],[@ElUniversalCtg]
8,Top Series Netflix 🇨🇴 Mie19May\n\n1. Pablo Esc...,2021-05-19 20:37:27,Bogotá,"Bogotá, D.C., Colombia",1395116413237485570,https://twitter.com/i/web/stuatus/139511641323...,Top Series Netflix 🇨🇴 Mie19May\n\n1. Pablo Esc...,[],[],[]
9,Por circunstancias de la vida anoche me vi la ...,2021-05-19 17:55:22,Bogotá,"Bogotá, D.C., Colombia",1395075623438241796,https://twitter.com/i/web/stuatus/139507562343...,Por circunstancias de la vida anoche me vi la ...,[],[],[]


In [20]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  
  

data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()


,tweets,fecha,ciudad,localizacion,id,enlace,texto_limpio,links,hashtags,mentions,emojis
0,@MarceloMejia Que bien. Y la reina del flow? \...,2021-05-20 17:48:43,Bogotá,Girardot Cundinamarca,1395436339328430080,https://twitter.com/i/web/stuatus/139543633932...,Que bien. Y la reina del flow? \n\n,[https://t.co/YLhSYjCaFQ],[],[@MarceloMejia],[]
1,Es normal tener pegadas las canciones de la re...,2021-05-20 15:10:53,Bogotá,Bogota,1395396619437215750,https://twitter.com/i/web/stuatus/139539661943...,Es normal tener pegadas las canciones de la re...,[],[],[],[]
2,Carolina Ramírez se confesó y reveló si ha vis...,2021-05-20 08:49:40,Bogotá,Bogotá,1395300682958245895,https://twitter.com/i/web/stuatus/139530068295...,Carolina Ramírez se confesó y reveló si ha vis...,[https://t.co/XHtCzLIciE],[],[],[]
3,@muchotropico La reina del flow?,2021-05-20 02:52:22,Bogotá,"Bogotá, D.C., Colombia",1395210765121036290,https://twitter.com/i/web/stuatus/139521076512...,La reina del flow?,[],[],[@muchotropico],[]
4,La verdad no le tenia fe a la reina del flow 2...,2021-05-20 02:33:31,Bogotá,"Bogotá, D.C., Colombia",1395206021812047875,https://twitter.com/i/web/stuatus/139520602181...,La verdad no le tenia fe a la reina del flow 2...,[],[],[],"[🔪, 🔪, 🔪]"


In [27]:
data['hora'] = data['fecha'].dt.floor('H').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp


,hora,cnt
16,00:00:00,1
1,01:00:00,6
0,02:00:00,13
5,03:00:00,3
3,04:00:00,4
12,05:00:00,1
15,08:00:00,1
18,10:00:00,1
10,11:00:00,2
17,12:00:00,1


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [32]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)


In [33]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_sp = stopwords.words('spanish')

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto

tfidf_vect = TfidfVectorizer(preprocessor=pre_procesado)
tfidf = tfidf_vect.fit_transform(data.texto_limpio.values)

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

voc = [k for k,v in sorted(tfidf_vect.vocabulary_.items(), key=lambda kv: kv[1])]
temp = pd.DataFrame(tfidf.toarray())
temp = cosine_similarity(temp.values)
temp = pd.DataFrame(temp)
# temp.columns = data.tweet
# temp.index = data.tweet
temp

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,1.000000,0.061372,0.039331,0.354664,0.053830,0.026861,0.142013,0.000000,0.034372,0.031701,...,0.034144,0.037335,0.066314,0.000000,0.051571,0.035127,0.036522,0.040354,0.142013,0.435705
1,0.061372,1.000000,0.019190,0.173043,0.026264,0.013105,0.069289,0.000000,0.016770,0.015467,...,0.016659,0.018216,0.032355,0.000000,0.025162,0.017139,0.017820,0.019689,0.069289,0.026740
2,0.039331,0.019190,1.000000,0.110898,0.016832,0.008399,0.044405,0.000000,0.010747,0.009912,...,0.010676,0.092808,0.020735,0.000000,0.352341,0.010984,0.011420,0.071723,0.044405,0.017137
3,0.354664,0.173043,0.110898,1.000000,0.151778,0.075735,0.400416,0.000000,0.096913,0.089384,...,0.096272,0.105267,0.186978,0.000000,0.145408,0.099044,0.102977,0.113780,0.400416,0.154529
4,0.053830,0.026264,0.016832,0.151778,1.000000,0.011495,0.060774,0.000000,0.014709,0.013566,...,0.014612,0.015977,0.028379,0.000000,0.022070,0.015033,0.015630,0.017269,0.060774,0.023454
5,0.026861,0.013105,0.008399,0.075735,0.011495,1.000000,0.030326,0.000000,0.007340,0.006770,...,0.007291,0.007972,0.014161,0.000000,0.011013,0.007501,0.007799,0.008617,0.030326,0.011703
6,0.142013,0.069289,0.044405,0.400416,0.060774,0.030326,1.000000,0.000000,0.038806,0.035791,...,0.038549,0.042151,0.074869,0.000000,0.058224,0.039659,0.041234,0.045559,1.000000,0.061876
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.034372,0.016770,0.010747,0.096913,0.014709,0.007340,0.038806,0.000000,1.000000,0.008662,...,0.784557,0.010202,0.018121,0.000000,0.014092,0.807149,0.009980,0.011027,0.038806,0.014976
9,0.031701,0.015467,0.009912,0.089384,0.013566,0.006770,0.035791,0.000000,0.008662,1.000000,...,0.008605,0.009409,0.016713,0.000000,0.012997,0.008853,0.009205,0.010170,0.035791,0.013812


In [35]:
trace = go.Heatmap(z=temp.values.tolist(),
                   x=temp.index.values,
                   y=temp.columns.values,
                   colorscale='Oranges')

layout = go.Layout(title='🔥 Mapa de calor entre tweets 🔥',
                   width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)

In [36]:
import scipy.cluster.hierarchy as sch
import numpy as np

pairwise_distances = sch.distance.pdist(temp)
linkage = sch.linkage(pairwise_distances, method='ward')
idx_to_cluster_array = sch.fcluster(linkage, pairwise_distances.max() * 0.5, criterion='distance')
idx = np.argsort(idx_to_cluster_array)
temp = temp.copy()
    
temp2 = temp.iloc[idx, :].T.iloc[idx, :]
my_idx = idx_to_cluster_array

temp2.index = ['tweet #'+str(i) for i in temp2.columns]
temp2.columns = temp2.index

In [37]:
trace = go.Heatmap(z=temp2.values.tolist(),
                   x=temp2.index.values,
                   y=temp2.columns.values,
                   colorscale='Oranges')

layout = go.Layout(title='🔥 Mapa de calor entre tweets (ordenado) 🔥',
                  width=800, height=800)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)